# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import torch
import numpy as np
import multiprocessing as mp
from torch.distributions import Normal
import torch.nn.functional as F
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline



Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='/home/frank/deep-reinforcement-learning/p2_continuous-control/Reacher_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action

action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print(env_info.agents)

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]
[932, 944, 946, 962, 980, 990, 1018, 1032, 1044, 1054, 1086, 1090, 1094, 1106, 1128, 1138, 1170, 1188, 1194, 1208]


In [5]:
env_info=env.reset(train_mode=True)[brain_name]
from PPOModel import Policy, Value
from PPOAgent2 import PPO_Agent

In [6]:
agent = PPO_Agent(state_size=33, action_size=4, random_seed=20, n_actors=len(env_info.agents))
GAMMA = .99
LAMBDA = .90
Epsilon = 0.2
#Beta = .02
TMAX=600
TAU = 1e-3 
MINI_BATCH = 50
BATCH_SIZE  = 600
iterations=10

In [ ]:
def TrainPPO(eps,tmax,episodes = 2000):
    for e in range(episodes):
        
        env_info = env.reset(train_mode=True)[brain_name]
        scores_list = []
        scores_window = deque(maxlen=100) 
        scores = np.zeros(agent.n_actors)
        states = env_info.vector_observations  
        state_list = []
        action_list = []
        probability_list = []
        entropy_list = []
        value_list = []
        reward_list = []
        for t in range(tmax):
            state_list.append(states)
            states = torch.tensor(states, dtype=torch.float)
            dist = agent.policy_net_new.forward(states)
            actions = dist.sample()
            value = agent.value_target.forward(states)
            value_list.append(value)
            action_list.append(actions)
            probs = dist.log_prob(actions)
            probability_list.append(probs)
            env_info = env.step(actions.cpu().detach().numpy())[brain_name]
            next_states = env_info.vector_observations  
            rewards = env_info.rewards
            reward_list.append(rewards)
            dones = env_info.local_done
            scores += env_info.rewards
            states = next_states
            
            
            if np.any(dones):
                break
        
        states_t =torch.tensor(state_list,dtype=torch.float)
        value_est = agent.Value_Estimation(reward_list,GAMMA)
        
        advantages,rets = agent.Advantage_Estimation(value_list,reward_list,GAMMA,LAMBDA,TMAX)
        advantages, rets = advantages.detach().numpy(),rets.detach().numpy()
        scores_window.append(scores)
        scores_list.append(scores)
       
        for i in range(iterations):
            for _ in range(BATCH_SIZE//MINI_BATCH):
                batch=agent.MakeMinibatch(state_list,value_list,value_est,\
                                       probability_list,advantages,rets,TMAX,MINI_BATCH)
                agent.Learn(batch.get("states"),batch.get("values"),batch.get("value_est")\
                            ,batch.get("probs"),batch.get("advs"),batch.get("returns"),Epsilon,MINI_BATCH)
                batch.clear()
            

                        
       
        

#        agent.NetworkUpdate(agent.policy_net_new, agent.policy_net_old)

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_window)), end="")
        if e % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_window)))
        if np.mean(scores_window)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e-100, np.mean(scores_window)))
            torch.save(agent.policy_net_old.state_dict(), 'PPOPolicycheckpointOld.pth')
            torch.save(agent.policy_net_new.state_dict(), 'PPOPolicycheckpointNew.pth')
            torch.save(agent.value_target.state_dict(), 'PPOValuecheckpoint.pth')
            break
        
    
    return scores



            
            
                

scores = TrainPPO(Epsilon,TMAX)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
    


Episode 0	Average Score: 0.24
Episode 56	Average Score: 0.30

In [ ]:
env.close()